In [40]:
from medmnist import ChestMNIST
from PIL import Image
from torchvision import transforms
import torchvision.models as models
import numpy as np
import tqdm
import torch
from sklearn.preprocessing import MinMaxScaler

In [19]:
# Load the ChestMNIST dataset
dataset = ChestMNIST(split="train", download=True, as_rgb=True, size=224)

images = dataset.imgs
labels = dataset.labels

Using downloaded and verified file: /Users/thollenbeak/.medmnist/chestmnist_224.npz


In [20]:
model = models.squeezenet1_1(weights=models.SqueezeNet1_1_Weights.DEFAULT, progress=True)

In [39]:
preprocess = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Normalize the image data to ensure the standardization values of the pre-trained model is appropriate
flattened_images = [image.ravel() for image in images]

scaler = MinMaxScaler()
normalized_images = scaler.fit_transform(images)

images = [image.reshape(224, 224) for image in normalized_images]


transformed_images = []

for image in tqdm.tqdm(images):
    image = Image.fromarray(image)
    transformed_images.append(preprocess(image))

input_tensor = torch.stack(transformed_images)

ValueError: Found array with dim 3. MinMaxScaler expected <= 2.